In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os, sys, h5py, time
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mat
import seaborn as sb
import tensorflow as tf
import scipy

import sys
sys.path.append('../../../..')
import mutagenesisfunctions as mf
import bpdev as bd
import helper
from deepomics import neuralnetwork as nn
from deepomics import utils, fit, visualize, saliency

import contacts

from Bio import AlignIO
import time as time
import pandas as pd
np.random.seed(42)

In [6]:
families = ['RF00023', 'RF00504']
trainsizes = ['2000', '1000', '800', '300']

In [19]:
for fam in families:
    for ts in trainsizes:
        arrayspath = 'mlp_512_%s_ts%s.npy'%(fam,ts)
        hol_mut2 = np.load(arrayspath)
        seqlen,_, dims,_ = hol_mut2.shape
        C = bd.get_wc(arrayspath, seqlen, dims, bpugSQ=0, denoise='APC')

        #get coordinates
        C[np.tril_indices(seqlen)] = np.min(C)
        bp_stretch = np.ravel(C)
        minval = np.sort(bp_stretch)[::-1][-1] #just gets the minimum value - can be changed to a different idx
        bs = np.where(C > minval)
        idx = np.argsort(C[bs])
        x = bs[0][idx] #sorts the x and y coords in order from lowest to highest so highest scores will be plotted last
        y = bs[1][idx]

        score_col = [C[xx,yy] for xx,yy in zip(x,y)] #gets the scores for each coordinate in the needed order

        #get the colour map to select colors for plotting
        pal = 'Blues'
        cmap = mat.cm.get_cmap(pal)


        markersize = 20

        #plot
        fig = plt.figure(figsize=(5,5))
        #plt.scatter(np.tril_indices(seqlen)[0], np.tril_indices(seqlen)[1], c=np.asarray([cmap(0.)]), s=markersize) #plot background 
        plt.scatter(x, y, c=score_col, cmap=pal, label = 'SoM', vmin=0., s=markersize)
        plt.scatter(y, x, c=score_col, cmap=pal, label = 'SoM', vmin=0., s=markersize)
        #plt.xlabel('Sequence Position', fontsize=10)
        #if ts == '300':
        #    plt.ylabel('Sequence Position', fontsize=10)
        #    plt.xticks(fontsize=10)
        plt.yticks(fontsize=10)

        ax = plt.gca()
        ax.invert_yaxis()
        ax.set_facecolor(cmap(0.))
        if ts == '300':
            ax.tick_params(axis='x', which='major', labelsize=10)
        if ts != '300':
            ax.set_xticklabels([])

        plt.savefig('%s_%s.png'%(fam,ts))
        #plt.show()
        plt.close()

In [17]:
plt.close('all')